In [21]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score


In [ ]:
class Dataset:
    def __init__(self, file_name):
        super().__init__()
        self.file_name = file_name

        df = pd.read_csv(f'../../DL_data/dataset/{self.file_name}')
        X = df.drop('churn', axis=1)
        y = df['churn']

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X, y, random_state=42, test_size=0.2,
                                                                              stratify=y)

    def get_train_dataset(self):
        train_dataset = Train_Dataset(self.X_train, self.y_train)
        val_dataset = Train_Dataset(self.X_val, self.y_val)

        return train_dataset, val_dataset


# Dataset
class Train_Dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    # getter
    def __getitem__(self, idx):
        data = self.data.iloc[idx].to_numpy()
        label = self.label.iloc[idx]

        return data, label

In [33]:
class MLP_model(nn.Module):
    def __init__(self, Cin):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(Cin, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)


In [26]:

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0, path='best_model.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.best = None
        self.count = 0
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best is None or (self.best - val_loss) > self.min_delta:
            self.best = val_loss
            self.count = 0
            torch.save(model.state_dict(), self.path)  # 베스트 갱신 시 저장
        else:
            self.count += 1
            if self.count >= self.patience:
                self.early_stop = True

In [34]:

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    dataset = Dataset(file_name='re_log_model_preprocessed.csv')
    train_dataset, val_dataset = dataset.get_train_dataset()

    Cin = train_dataset.__getitem__(0)[0].shape
    print(f"num Feature : {Cin}")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=32)

    model = MLP_model(Cin[0]).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_epoch = 100
    train_losses, val_losses, val_accuracies = [], [], []
    best_metrics = {'f1': 0, 'precision': 0, 'recall': 0, 'accuracy': 0}

    # Early Stopping
    es = EarlyStopping(patience=10, min_delta=0.0, path='../best_model.pt')

    for epoch in range(train_epoch):
        # 학습
        model.train()
        train_loss = 0.0
        for batch_data, batch_label in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{train_epoch}] Training"):
            batch_data = batch_data.to(device).float()
            batch_label = batch_label.to(device).float().unsqueeze(-1)

            optimizer.zero_grad()
            outputs = model(batch_data)                 # logits
            loss = criterion(outputs, batch_label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # 검증
        model.eval()
        val_loss = 0.0
        correct, total = 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for batch_data, batch_label in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{train_epoch}] Validation"):
                batch_data = batch_data.to(device).float()
                batch_label = batch_label.to(device).float().unsqueeze(-1)

                logits = model(batch_data)
                loss = criterion(logits, batch_label)
                val_loss += loss.item()

                probs = torch.sigmoid(logits)
                preds = (probs > 0.5).float()

                correct += (preds == batch_label).sum().item()
                total += batch_label.size(0)

                all_preds.extend(preds.cpu().numpy().ravel())
                all_labels.extend(batch_label.cpu().numpy().ravel())

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_accuracy = 100 * correct / total
        val_accuracies.append(val_accuracy)

        val_f1       = f1_score(all_labels, all_preds, zero_division=0)
        val_precision= precision_score(all_labels, all_preds, zero_division=0)
        val_recall   = recall_score(all_labels, all_preds, zero_division=0)

        print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.2f}%")
        print(f"Epoch {epoch+1} - F1 : {val_f1:.4f}, Precision : {val_precision:.4f}, Recall : {val_recall:.4f}")

        # 베스트 메트릭 갱신
        if val_accuracy > best_metrics['accuracy']:
            best_metrics.update({'f1': val_f1, 'precision': val_precision, 'recall': val_recall, 'accuracy': val_accuracy})

        # Early Stopping 체크: 검증 손실 계산 '직후'
        es(avg_val_loss, model)
        if es.early_stop:
            print("* Early stopping triggered.")
            break

    # 학습 종료 후: 베스트 가중치 로드
    model.load_state_dict(torch.load('../best_model.pt', map_location=device))
    model.eval()

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, digits=4))

    print("\n최종 평가 결과")
    print(f"Precision    : {best_metrics['precision']:.4f}")
    print(f"Recall       : {best_metrics['recall']:.4f}")
    print(f"F1 Score     : {best_metrics['f1']:.4f}")
    print(f"Accuracy (%) : {best_metrics['accuracy']:.2f}%")

if __name__ == '__main__':
    main()

Using device: cuda
num Feature : (13,)


[Epoch 1/100] Validation: 100%|██████████| 450/450 [00:01<00:00, 356.74it/s]


Train Loss : 0.2333 || Valid Loss : 0.2074 | Valid Accuracy : 93.41%
Epoch 1 - F1 : 0.9400, Precision : 0.9542, Recall : 0.9262


[Epoch 2/100] Validation: 100%|██████████| 450/450 [00:01<00:00, 362.08it/s]


Train Loss : 0.2140 || Valid Loss : 0.2065 | Valid Accuracy : 93.52%
Epoch 2 - F1 : 0.9410, Precision : 0.9545, Recall : 0.9278


...

Train Loss : 0.1997 || Valid Loss : 0.1980 | Valid Accuracy : 93.67%
Epoch 52 - F1 : 0.9424, Precision : 0.9552, Recall : 0.9300


[Epoch 53/100] Validation: 100%|██████████| 450/450 [00:01<00:00, 361.78it/s]


Train Loss : 0.1998 || Valid Loss : 0.1981 | Valid Accuracy : 93.73%
Epoch 53 - F1 : 0.9428, Precision : 0.9586, Recall : 0.9275


[Epoch 54/100] Validation: 100%|██████████| 450/450 [00:01<00:00, 352.98it/s]

Train Loss : 0.1999 || Valid Loss : 0.1964 | Valid Accuracy : 93.71%
Epoch 54 - F1 : 0.9427, Precision : 0.9581, Recall : 0.9277
* Early stopping triggered.

Classification Report:
              precision    recall  f1-score   support

         0.0     0.9126    0.9490    0.9304      6369
         1.0     0.9581    0.9277    0.9427      8010

    accuracy                         0.9371     14379
   macro avg     0.9353    0.9383    0.9365     14379
weighted avg     0.9379    0.9371    0.9372     14379


최종 평가 결과
Precision    : 0.9570
Recall       : 0.9291
F1 Score     : 0.9429
Accuracy (%) : 93.73%


# learning_rate  = 0.1, 0.001, 0.0001 비교

In [36]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    dataset = Dataset(file_name='re_log_model_preprocessed.csv')
    train_dataset, val_dataset = dataset.get_train_dataset()

    Cin = train_dataset.__getitem__(0)[0].shape
    print(f"num Feature : {Cin}")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=32)

    # 🔹 테스트할 학습률 리스트
    learning_rates = [0.001, 0.01, 0.1]
    results = []  # 각 학습률별 성능 저장 리스트

    for lr in learning_rates:
        print(f"\n==============================")
        print(f"학습률 {lr} 실험 시작")
        print(f"==============================")

        model = MLP_model(Cin[0]).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        train_epoch = 100
        best_metrics = {'f1': 0, 'precision': 0, 'recall': 0, 'accuracy': 0}

        es = EarlyStopping(patience=10, min_delta=0.0, path=f'../best_model_lr{lr}.pt')

        for epoch in range(train_epoch):
            # 학습
            model.train()
            train_loss = 0.0
            for batch_data, batch_label in train_loader:
                batch_data = batch_data.to(device).float()
                batch_label = batch_label.to(device).float().unsqueeze(-1)

                optimizer.zero_grad()
                outputs = model(batch_data)
                loss = criterion(outputs, batch_label)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            avg_train_loss = train_loss / len(train_loader)

            # 검증
            model.eval()
            val_loss = 0.0
            correct, total = 0, 0
            all_preds, all_labels = [], []

            with torch.no_grad():
                for batch_data, batch_label in val_loader:
                    batch_data = batch_data.to(device).float()
                    batch_label = batch_label.to(device).float().unsqueeze(-1)

                    outputs = model(batch_data)
                    loss = criterion(outputs, batch_label)
                    val_loss += loss.item()
                    probs = torch.sigmoid(outputs)
                    preds = (probs > 0.5).float()

                    correct += (preds == batch_label).sum().item()
                    total += batch_label.size(0)
                    all_preds.extend(preds.cpu().numpy().ravel())
                    all_labels.extend(batch_label.cpu().numpy().ravel())

            avg_val_loss = val_loss / len(val_loader)
            val_accuracy = 100 * correct / total
            val_f1 = f1_score(all_labels, all_preds)
            val_precision = precision_score(all_labels, all_preds)
            val_recall = recall_score(all_labels, all_preds)

            print(f"[LR {lr}] Epoch {epoch+1:03d} | "
                  f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")

            # EarlyStopping
            es(avg_val_loss, model)
            if es.early_stop:
                print(f"* Early Stopping triggered for lr={lr}")
                break

            # Best metrics 갱신
            if val_accuracy > best_metrics['accuracy']:
                best_metrics.update({
                    'f1': val_f1, 'precision': val_precision,
                    'recall': val_recall, 'accuracy': val_accuracy
                })

        # 학습 완료 후 결과 저장
        results.append({
            'lr': lr,
            'f1': best_metrics['f1'],
            'precision': best_metrics['precision'],
            'recall': best_metrics['recall'],
            'accuracy': best_metrics['accuracy']
        })

    # ---------------- 결과 비교 출력 ----------------
    print("\n< 학습률별 최종 성능 비교 >")
    print("{:<10}{:<10}{:<10}{:<10}{:<10}".format("LR", "Accuracy", "F1", "Precision", "Recall"))
    for r in results:
        print("{:<10}{:<10.2f}{:<10.4f}{:<10.4f}{:<10.4f}".format(
            r['lr'], r['accuracy'], r['f1'], r['precision'], r['recall']
        ))

if __name__ == '__main__':
    main()


Using device: cuda
num Feature : (13,)

학습률 0.001 실험 시작
[LR 0.001] Epoch 001 | Train Loss: 0.2322 | Val Loss: 0.2074 | Val Acc: 93.46%
[LR 0.001] Epoch 002 | Train Loss: 0.2163 | Val Loss: 0.2049 | Val Acc: 93.50%
[LR 0.001] Epoch 003 | Train Loss: 0.2130 | Val Loss: 0.2041 | Val Acc: 93.54%
[LR 0.001] Epoch 004 | Train Loss: 0.2110 | Val Loss: 0.2057 | Val Acc: 93.53%
[LR 0.001] Epoch 005 | Train Loss: 0.2109 | Val Loss: 0.2044 | Val Acc: 93.50%
[LR 0.001] Epoch 006 | Train Loss: 0.2085 | Val Loss: 0.2027 | Val Acc: 93.46%
[LR 0.001] Epoch 007 | Train Loss: 0.2094 | Val Loss: 0.2008 | Val Acc: 93.62%
[LR 0.001] Epoch 008 | Train Loss: 0.2083 | Val Loss: 0.2017 | Val Acc: 93.62%
[LR 0.001] Epoch 009 | Train Loss: 0.2087 | Val Loss: 0.2010 | Val Acc: 93.57%
[LR 0.001] Epoch 010 | Train Loss: 0.2070 | Val Loss: 0.2009 | Val Acc: 93.53%
[LR 0.001] Epoch 011 | Train Loss: 0.2069 | Val Loss: 0.1997 | Val Acc: 93.58%
[LR 0.001] Epoch 012 | Train Loss: 0.2073 | Val Loss: 0.2008 | Val Acc: 93.

# ROC-AUC , PR-AUC

In [44]:
#ROC-AUC, PR-AUC  계산

import torch
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 데이터셋 불러오기
dataset = Dataset(file_name='re_log_model_preprocessed.csv')
train_dataset, val_dataset = dataset.get_train_dataset()

# DataLoader
from torch.utils.data import DataLoader
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Cin = train_dataset.__getitem__(0)[0].shape[0]

# 모델 구조 정의 동일해야함
import torch.nn as nn

class MLP_model(nn.Module):
    def __init__(self, Cin):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(Cin, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

# 모델 불러오기
model = MLP_model(Cin).to(device)
state_dict = torch.load("/best_model.pt", map_location=device)
model.load_state_dict(state_dict)
model.eval()
print("Model loaded successfully")

# 예측, AUC 계산
y_probs, y_trues = [], []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device).float()
        yb = yb.to(device).float()

        out = model(xb)
        prob = torch.sigmoid(out)
        y_probs.append(prob.detach().cpu().numpy().ravel())
        y_trues.append(yb.detach().cpu().numpy().ravel())


y_prob = np.concatenate(y_probs)
y_true = np.concatenate(y_trues)

roc_auc = roc_auc_score(y_true, y_prob)
pr_auc = average_precision_score(y_true, y_prob)

print(f"\nROC-AUC : {roc_auc:.4f}")
print(f"PR-AUC  : {pr_auc:.4f}")


Using device: cuda
Model loaded successfully

ROC-AUC : 0.9658
PR-AUC  : 0.9791
